<a href="https://colab.research.google.com/github/CalebEng/Ai-Project/blob/master/Ai_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
import torch
from torch import nn

!pip install --upgrade tiktoken
import tiktoken
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Ai work')
#torch.__version__
enc = tiktoken.encoding_for_model('gpt-3.5-turbo')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.9 MB/s eta 0:00:00
Mounted at /content/drive



------------------- Prep and loading data sets -------------------

--ONLY RUN THE NEXT IF CSV FILE CONTAINING CONVOS HAS A INDEX COLUMN--

In [ ]:
import pandas as pd
df = pd.read_csv('Conversation.csv')
# If you know the name of the column skip this
first_column = df.columns[0]
# Delete first
df = df.drop([first_column], axis=1)
df.to_csv('Conversation.csv', index=False)


In [ ]:
df.to_csv('Conversation_enc.csv',index=False)

In [16]:
test = torch.tensor(temp1)

ValueError: ignored

In [5]:
def format(x):
  y=enc.encode(x)
  return torch.tensor(y)

df['question'] = df['question'].map(format)
df['answer'] = df['answer'].map(format)


In [4]:
df = pd.read_csv('Conversation.csv')

In [ ]:
df

In [4]:

df2 = pd.read_csv('Conversation.csv')

questions = df2['question'].tolist()
answers = df2['answer'].tolist()

temp = map(format, questions)
temp2 = map(format, answers)

questions = list(temp)
answers = list(temp2)


--EXIT--

--Padding and turning data into tensors--

In [18]:
df = pd.read_csv('Conversation.csv')

temp1 = df['question'].tolist()
temp2 = df['answer'].tolist()


for i in range(0,len(temp1)):
  count= len(temp1[i].split())
  if count<20:
    for x in range(count, 25):
        temp1[i]+=" -"

for i in range(0,len(temp2)):
  count= len(temp1[i].split())
  if count<20:
    for x in range(count, 25):
        temp1[i]+=" -"




In [21]:
def format(x):
  y=enc.encode(x)
  return torch.tensor(y)

qT = map(format, temp1)
aT = map(format, temp2)

questions = list(qT)
answers = list(aT)

In [ ]:
questions


In [7]:
n = int(0.9*len(questions))

train_data_q = questions[:n]
train_data_a = answers[:n]

val_data_q = questions[n:]
val_data_a = answers[n:]

In [8]:
block_size = 8


In [ ]:
x = train_data_q[:block_size]
y = train_data_a[:block_size]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target is: {target}")

In [ ]:
batch_size = 4
block_size = 8

